<a href="https://colab.research.google.com/github/Tom-Lennox/Kaggle/blob/master/titanic_20200428.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# title
# 【result】0.65071

In [1]:
# kaggle APIセット
!pip install kaggle

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
# 「Download 100%.」と表示で成功。

# [kaggle.json]を持参する。
from google.colab import drive
drive.mount('/content/drive')

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c titanic

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c titanic
!ls

Download 100%.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
cp: cannot create regular file '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, s

In [33]:
# ▼ all_data作成
train = pd.read_csv("/content/train.csv")
test_x = pd.read_csv("/content/test.csv")

train_x = train.drop("Survived",axis=1)
train_y = train["Survived"]
test_x = pd.read_csv("/content/test.csv")

train_x['label'] = 'train'
test_x['label'] = 'test'
all_data = pd.concat([train_x,test_x],axis=0,sort=False)
#学習データとテストデータを統合、カラムsortされると困る。

print('▼ 欠損　確認')
print(all_data.isnull().sum()[all_data.isnull().sum() > 0].index.tolist())

print('▼ 欠損　個数')
print(all_data.isnull().sum()[all_data.isnull().sum() > 0])

print('▼ 欠損　data')
print(all_data['Fare'][all_data['Fare'].isnull() == True])

print('▼ 補完')
all_data['Age'].fillna(all_data.Age.mean(), inplace=True)
all_data['Embarked'] = all_data['Embarked'].fillna("None")
all_data['Cabin'] = all_data['Cabin'].fillna("None")
all_data['Fare'].fillna(all_data.Fare.mean(), inplace=True)
# object, intに分けて自動処理。

print('▼ 補完　確認')
print(all_data.isnull().sum()[all_data.isnull().sum() > 0])

# train['Age'].fillna(train.Age.mean(), inplace=True)
all_data[['Age', 'Cabin', 'Embarked', 'Fare']]
# ▲ all_data作成

▼ 欠損　確認
['Age', 'Fare', 'Cabin', 'Embarked']
▼ 欠損　個数
Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64
▼ 欠損　data
152   NaN
Name: Fare, dtype: float64
▼ 補完
▼ 補完　確認
Series([], dtype: int64)


,Age,Cabin,Embarked,Fare
0,22.000000,None,S,7.2500
1,38.000000,C85,C,71.2833
2,26.000000,None,S,7.9250
3,35.000000,C123,S,53.1000
4,35.000000,None,S,8.0500
...,...,...,...,...
413,29.881138,None,S,8.0500
414,39.000000,C105,C,108.9000
415,38.500000,None,S,7.2500
416,29.881138,None,S,8.0500


In [34]:
combine1 = [all_data]

for all_data in combine1: 
  all_data['Salutation'] = all_data.Name.str.extract(' ([A-Za-z]+).', expand=False) 
for all_data in combine1: 
  all_data['Salutation'] = all_data['Salutation'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
  all_data['Salutation'] = all_data['Salutation'].replace('Mlle', 'Miss')
  all_data['Salutation'] = all_data['Salutation'].replace('Ms', 'Miss')
  all_data['Salutation'] = all_data['Salutation'].replace('Mme', 'Mrs')
Salutation_mapping ={"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 
for tain in combine1:
  all_data['Salutation'] = all_data['Salutation'].map(Salutation_mapping)
  all_data['Salutation'] = all_data['Salutation'].fillna(0)
  print(all_data['Salutation'].unique())
  del all_data['Salutation']
  del all_data['Name']

# ▲
# ▼ Ticket
for all_data in combine1:
  all_data['ticket_left'] = all_data['Ticket'].apply(lambda x: str(x)[0])
  # strに変換 + 1文字目を取得
  all_data['ticket_left'] = all_data['ticket_left'].apply(lambda x: str(x))
  all_data['del_ticket_left_zero'] = np.where((all_data['ticket_left']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0')
  all_data['ticket_left'] = np.where((all_data['ticket_left']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), all_data['ticket_left'], all_data['del_ticket_left_zero'])
  del all_data['del_ticket_left_zero']
  all_data['tcker_len'] = all_data['Ticket'].apply(lambda x: len(x))
  del all_data['Ticket']
all_data['ticket_left']=all_data['ticket_left'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3)
print('▼ print(all_data["ticket_left"])')
print(all_data['ticket_left'].unique())
# ▲ Ticket
# ▼ Cabin
for all_data in combine1:
  all_data['cabin_left'] = all_data['Cabin'].apply(lambda x: str(x)[0])
  all_data['cabin_left'] = all_data['cabin_left'].apply(lambda x: str(x))
  # print(all_data['cabin_left'].unique())
  # ▲ コピペ用
  all_data['del_cabin_left_zero'] = np.where((all_data['cabin_left']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']), '0','0')
  # ▲ ！手直しの必要有。
  all_data['cabin_left'] = np.where((all_data['cabin_left']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']), all_data['cabin_left'], all_data['del_cabin_left_zero'])
  # del all_data['Cabin']
  del all_data['del_cabin_left_zero']
  del all_data['Cabin']
all_data['cabin_left']=all_data['cabin_left'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)
# ▲ ！手直しの必要有。
# ▲ Cabin
# ▼ family_of　（新規追加）
all_data['family_of'] = all_data['SibSp'] + all_data['Parch'] + 1
for all_data in combine1:
  all_data['is_alone'] = 0
  all_data.loc[all_data['family_of'] == 1, 'is_alone'] = 1
# ▲ family_of　（新規追加）
# ▼ train
del all_data['Embarked']
del all_data['Sex']
#カテゴリ変数となっているカラムを取り出す
# cal_list = all_data.dtypes[all_data.dtypes=="object"].index.tolist()

#カテゴリ変数をget_dummiesによるone-hot-encodingを行う
# all_data = pd.get_dummies(all_data,columns=cal_list)

# train_x = all_data.iloc[:train_x.shape[0],:].reset_index(drop=True)
# test_x = all_data.iloc[train_x.shape[0]:,:].reset_index(drop=True)

# Split your data
train_x = all_data[all_data['label'] == 'train']
test_x = all_data[all_data['label'] == 'test']

# Drop your labels
train_x = train_x.drop('label', axis=1)
test_x = test_x.drop('label', axis=1)
# 分割

print("train_x: "+str(train_x.shape))
print("test_x: "+str(test_x.shape))
#サイズを確認

print(train_x.isnull().sum()[train_x.isnull().sum() > 0])
print(test_x.isnull().sum()[test_x.isnull().sum() > 0])
# ▲ 

[1. 3. 2. 4. 0. 5.]
▼ print(all_data["ticket_left"])
[3 0 1 2]


In [18]:
print('▼ train')
train_x.info()
print('▼ test')
test_x.info()

▼ train
<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Age          891 non-null    float64
 3   SibSp        891 non-null    int64  
 4   Parch        891 non-null    int64  
 5   Fare         891 non-null    float64
 6   ticket_left  891 non-null    int64  
 7   tcker_len    891 non-null    int64  
 8   cabin_left   891 non-null    int64  
 9   family_of    891 non-null    int64  
 10  is_alone     891 non-null    int64  
dtypes: float64(2), int64(9)
memory usage: 83.5 KB
▼ test
<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Age  

In [0]:
from sklearn.ensemble import RandomForestClassifier

clf = svm.SVC()
clf.fit(train_x,train_y)
prediction = clf.predict(test_x)

#ランダムフォレストによる分類
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_x, train_y)
prediction = random_forest.predict(test_x)

submission = pd.DataFrame({
  "PassengerId": test_x["PassengerId"],
  "Survived": prediction
})
submission.to_csv('result.csv', index=False)     
files.download('result.csv')